# Coordinate Transformations

## About coordinate transformations

There are a number of essential coordinate transformations specific to neutron-scattering such as conversion from time-of-flight to $\lambda$ (wavelength) or $Q$ (momentum transfer).
Such coordinate transformations are also frequently referred to as "unit conversion", but here we avoid this terminology, to clearly distinguish from [conversion of array units using](https://scipp.github.io/generated/functions/scipp.to_unit.html#scipp.to_unit) `sc.to_unit`.

Scipp provides [coordinate transformations using](https://scipp.github.io/user-guide/coordinate-transformations.html) `sc.transform_coords`.
Scippneutron defines concrete transformations for time-of-flight neutron scattering, as well as building blocks for customizing transformations.
Both are discussed in the following.

## Built-in transformations

### Overview of time-of-flight transformations

For background, we refer to PaN-learning on [Introduction to neutron scattering](https://e-learning.pan-training.eu/wiki/index.php/Introduction_to_neutron_scattering) and [Basics of neutron scattering](https://e-learning.pan-training.eu/wiki/index.php/Basics_of_neutron_scattering).
Below we also describe the more direct approach.

We define the beamline geometry as in this image:

![image](../_static/beamline/beamline_coordinates_light.svg)

Where the source position defines the point where $t=0$ (or vice versa).
And $2\theta$ is the scattering angle, as defined in Bragg's law, not to be confused with $\theta_{\mathsf{spherical}}$ in sperical coordinates.
In many instruments the beam is roughly aligned with the $z$-axis, so $2\theta = \theta_{\mathsf{spherical}}$.
The factor of $2$ is a recurrent source of bugs.

In addition:

- $h$ is the Planck constant, $m_{\mathsf{n}}$ is the neutron mass.
- $d$ is the interplanar lattice spacing
- $\lambda$ is the de Broglie wavelength
- $\vec{Q}$ (`Q_vec` in code) is the momentum transfer and $Q$ its norm
- $E$ is the energy

In the special case of inelastic neutron scattering we have additionally:

- $E_i$ is the known incident energy. This case is called *direct inelastic scattering*.
  We define $t_0 = \sqrt{L_1^2 \cdot m_{\mathsf{n}} / (2 E_i)}$
- $E_f$ is the known final energy. This case is called *indirect inelastic scattering*.
  We define $t_0 = \sqrt{L_2^2 \cdot m_{\mathsf{n}} / (2 E_f)}$
  In this case $E_f$ is actually determined by analyzer crystals in the secondary flight path.
  It is assumed that the detector positions are set to the effective (neutronic) rather than physical positions, since the computation of $L_2$ assumes a straight line connection between sample and detectors.

Conversions between these quantities are given by the following table:

|Input coord |Output coord |Equation used for coord transformation |
|---|---|:---|
|`tof`|`dspacing`|$d = \frac{\lambda}{2\sin(\theta)} = \frac{h \cdot t}{L_{\mathsf{total}} \cdot m_{\mathsf{n}} \cdot 2 \sin(\theta)}$|
| `tof`|`wavelength`|$\lambda = \frac{h \cdot t}{m_{\mathsf{n}} \cdot L_{\mathsf{total}}}$|
|`wavelength`|`Q`|$Q = \frac{4 \pi \sin(\theta)}{\lambda}$|
|`Q`|`wavelength`|$\lambda = \frac{4 \pi \sin(\theta)}{Q}$|
|`wavelength`|`Q_vec`|$\vec{Q} = \vec{k}_i - \vec{k}_f = \frac{2\pi}{\lambda} \left(\hat{e}_i - \hat{e}_f\right)$|
| `tof`|`energy`|$E = \frac{m_{\mathsf{n}}L^2_{\mathsf{total}}}{2t^2}$|
| `tof`|`energy_transfer` (direct)|$E = E_i - \frac{m_{\mathsf{n}}L^2_{\mathsf{2}}}{2\left(t-t_0\right)^{2}}$|
| `tof`|`energy_transfer` (indirect)|$E = \frac{m_{\mathsf{n}}L^2_{\mathsf{1}}}{2\left(t-t_0\right)^{2}} - E_f$|

See the reference of [scippneutron.conversion](../generated/modules/scippneutron.conversion.rst) for additional conversions and more details.

### Transformation graphs

The above table defines how an output coordinate such as `wavelength` is computed from inputs such as `Ltotal` and `tof`.
In practice not all input data directly comes with all the required metadata.
For example, instead of `Ltotal` the metadata may only contain `source_position`, `sample_position`, and `position`.
These can then be used to compute derived metadata such as `Ltotal`.
`transform_coords` can automatically do this by walking a transformation graph.

The transformations typically require two components:

1. A definition of the beamline geometry which defines, e.g., how scattering angles are to be computed from positions of beamline components such as sample and detector positions.
2. A definition of the scattering kinematics and dynamics, e.g., how $\lambda$ or $Q$ can be computed from time-of-flight for an elastic scattering process.

Pre-defined standard components (transformation graphs and graph building blocks) are available in [scippneutron.conversion.graph](../generated/modules/scippneutron.conversion.graph.rst):

In [ ]:
import scipp as sc
from scippneutron.conversion import graph

`graph.beamline.beamline` defines a "simple" (straight) time-of-flight beamline with source-, sample-, and detector positions.
Two variants, with and without scattering are provided.

Without scattering, the transformation graph is simple.
This intended for use with monitors or imaging beamlines where no scattering from a sample occurs:

In [ ]:
sc.show_graph(graph.beamline.beamline(scatter=False), simplified=True)

The graph is to be read as follows.
Starting with the node for the desired output coordinate, here `Ltotal`, `transform_coords` proceeds as follows:

1. If `Ltotal` is found in the metadata, no transformation is performed.
   Return `Ltotal`.
2. If `Ltotal` is not found in the metadata,

   1. For each input to the node (arrows pointing *to* the node, here `source_position` and `position`)  go to 1., i.e., either find the input, or compute it by continuing to walk upwards in the graph.
   2. Compute `Ltotal` from the found or computed inputs and return it.
   
If the top of the graph is reached without finding all required inputs, the transformation fails.
Refer to the scipp documentation on [Coordinate Transformations](https://scipp.github.io/user-guide/coordinate-transformations.html) for a more detailed explanation.

A more complex example is `conversions.beamline` for a scattering process:

In [ ]:
sc.show_graph(graph.beamline.beamline(scatter=True), simplified=True)

In addition to defining how beamline parameters can be computed, a second building block defines how, e.g., $\lambda$ or $Q$ can be computed from an input coordinate.
There are four cases:

- `conversions.kinematic` for straight propagation of neutrons, i.e., without any scattering process.
  This is intended for use in combination with `conversions.beamline(scatter=False)`.
- `conversions.elastic` for elastic scattering processes.
- `conversions.direct_inelastic` for direct-inelastic scattering processes, i.e., for fixed incident energies.
- `conversions.indirect_inelastic` for direct-inelastic scattering processes, i.e., fixed final energies, typically using analyzer crystals.

The coordinate transformation logic defined by these four cases is as follows:

In [ ]:
sc.show_graph(graph.tof.kinematic("tof"), simplified=True)

In [ ]:
sc.show_graph(graph.tof.elastic("tof"), simplified=True)

In [ ]:
sc.show_graph(graph.tof.direct_inelastic("tof"), simplified=True)

In [ ]:
sc.show_graph(graph.tof.indirect_inelastic("tof"), simplified=True)

Computing energy transfer for an indirect geometry instrument requires changes to the `beamline` graph to account for scattering off of the analyzers.
This has not yet been implemented in ScippNeutron as the precise input coordinates for relevant instruments are unknown.
But an example has been developed for a summer school [here](https://ess-dmsc-dram.github.io/dmsc-school/4-reduction/qens-reduction.html).

### Performing a coordinate transformation

The transformation graphs depicted above can now be used to transform time-of-flight data to, e.g., `wavelength`.
We need to combine the two building blocks, the beamline transformation graph and the elastic scattering transformation graph, into a single graph:

In [ ]:
import scipp as sc
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

graph = {**beamline(scatter=True), **elastic("tof")}
sc.show_graph(graph, simplified=True)

We use the following powder diffraction data to demonstrate the conversion:

In [ ]:
import scippneutron as scn

da = scn.data.powder_sample()['data']
da

The result after applying the coordinate transformation is:

In [ ]:
da.transform_coords("wavelength", graph=graph)

Note how `transform_coords` automatically handles event data.

## Customizing transformations

Many neutron beamlines are more complex than what is assumed by the simple built-in transformation graphs.
The coordinate transformation mechanism is completely generic and is customizable.
We provide four examples:

1. [Gravity correction](#Gravity-correction) demonstrates how to replace a node in an existing graph.
   The example takes into account a gravity drop of scattered neutrons when computing $2\theta$.
1. [Plane wave beam](#Plane-wave-beam) demonstrates how to replace a node in an existing graph.
   The example computes $L_{\text{total}}$ not using a straight-line definition (which assumes a point-like source) but instead used the direction of the incident beam.
   This could be used for imaging beamlines to ensure that all image sensor pixels have identical wavelength coordinates.
1. [Diffraction calibration](#Diffraction-calibration) demonstrates a completely custom transformation without use of pre-defined graphs.
   The example uses calibration results to compute the interplanar lattice spacing $d$, instead of computing $d$ using positions of beamline components.
1. [2-D Rietveld](#2-D-Rietveld) demonstrates how a new node is added to an existing graph.
   The example computes $d_{\text{perp}}$ and $d$ as used in 2-D Rietveld.
   This also demonstrates how to continue working with data with multiple output coordinates using binning.

### Gravity correction

For techniques such as SANS which are probing low Q regions, the basic conversion approach may not be sufficient.
A computation of $2\theta$ may need to take into account gravity since the gravity drop after scattering is not negligible.
This can be achieved by replacing the function for computation of `two_theta` that is defined as part of `conversion.graph.beamline.beamline(scatter=True)`.

ScippNeutron provides [scn.conversion.beamline.scattering_angles_with_gravity](../generated/modules/scippneutron.conversion.beamline.scattering_angles_with_gravity.rst) which computes both the polar scattering angle `two_theta` and the azimuthal angle `phi`.
To insert it into the graph, first remove the default function for `two_theta` and then insert `scattering_angles_with_gravity`: 

In [ ]:
from scippneutron.conversion import beamline, graph

q_with_gravity = {**graph.beamline.beamline(scatter=True), **graph.tof.elastic_Q("tof")}
del q_with_gravity["two_theta"]
q_with_gravity[("two_theta", "phi")] = beamline.scattering_angles_with_gravity

The result is as follows:

In [ ]:
sc.show_graph(q_with_gravity, simplified=True)

We can use this to convert SANS data to $Q$.
Our custom transformation graph requires a `gravity` input coordinate, so we define one.
In this case (LARMOR beamline) "up" is along the `y` axis:

In [ ]:
import scippneutron as scn
from scipp.constants import g

da = scn.data.tutorial_dense_data()['detector']
# Convert to bin centers so we can later bin into Q bins
da.coords["tof"] = 0.5 * (da.coords["tof"]["tof", :-1] + da.coords["tof"]["tof", 1:])
da.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g
da_gravity = da.transform_coords("Q", graph=q_with_gravity)
da_gravity

As a final step we may then bin our data into $Q$ bins:

In [ ]:
q_bins = sc.linspace(dim="Q", unit="1/angstrom", start=0.0, stop=15.0, num=100)
da_gravity = da_gravity.flatten(to="Q").hist(Q=q_bins)
da_gravity.plot(norm="log")

### Plane wave beam

The built-in `conversion.graph.beamline.beamline(scatter=False)` assumes a straight line from source to detector, i.e., treats the source as a point.
If we use this to compute `wavelength` for an imaging beamline this will result in different wavelengths for each image sensor pixel, which may be undesirable.
We can define a custom `Ltotal` function to compute the average distance along the beam:

In [ ]:
import scipp as sc
from scippneutron.conversion import graph


def Ltotal(incident_beam, source_position, position):
    n = incident_beam / sc.norm(incident_beam)
    projected = sc.dot(position, n) * n
    return sc.norm(sc.mean(projected) - source_position)


plane_graph = {**graph.beamline.incident_beam(), **graph.tof.kinematic("tof")}
plane_graph["Ltotal"] = Ltotal
sc.show_graph(plane_graph, simplified=True)

Note how `Ltotal` uses the *mean* of the projected position to ensure that the coordinate transformation will produce a `wavelength` coordinate that does not depend on the pixel:

In [ ]:
import scippneutron as scn

da = scn.data.tutorial_dense_data()[
    'detector'
]  # Note that this is actually a SANS beamline, not an imaging beamline
da_plane = da.transform_coords("wavelength", graph=plane_graph)
da_plane

### Diffraction calibration

While the built-in conversion graphs support computation of $d$-spacing based on beamline component positions and time-of-flight, in practice it frequently is computed based parameters from a calibration file.
We define a function `dspacing` to compute this:

In [ ]:
import scipp as sc


def dspacing(tof, tzero, difc):
    difc = sc.reciprocal(difc)
    return difc * (tof - tzero)


calibration_graph = {"dspacing": dspacing}
sc.show_graph(calibration_graph)

We can then load powder data and set the required input coordinates from a corresponding calibration file:

In [ ]:
import scippneutron as scn

da = scn.data.powder_sample()['data']
calibration = scn.data.powder_calibration()
da.coords["tzero"] = calibration["tzero"].data
da.coords["difc"] = calibration["difc"].data

The coordinate transformation then yields:

In [ ]:
da.transform_coords("dspacing", graph=calibration_graph)

### 2-D Rietveld

Powder diffraction data is typically converted to $d$-spacing for Rietveld refinement.
A more advanced approach additionally requires data depending on $d_{\perp}$ for a [2-D Rietveld refinement process](https://journals.iucr.org/j/issues/2017/03/00/pd5090/index.html).
We define

In [ ]:
def dspacing_perpendicular(wavelength, two_theta):
    lambda_K = sc.scalar(1.0, unit="angstrom")
    return sc.sqrt(wavelength**2 - 2 * (lambda_K**2) * sc.log(sc.cos(0.5 * two_theta)))

We then add this as a new node (new dict item) to a graph consisting of built-ins:

In [ ]:
from scippneutron.conversion import graph

graph = {**graph.beamline.beamline(scatter=True), **graph.tof.elastic("tof")}
graph["d"] = "dspacing"  # rename to brief name
graph["d_perp"] = dspacing_perpendicular

The result is:

In [ ]:
import scipp as sc

del graph["energy"]  # not necessary, clarifies conversion graph
del graph["Q"]  # not necessary, clarifies conversion graph
sc.show_graph(graph, simplified=True)

We can then apply the transformation:

In [ ]:
import scippneutron as scn

da = scn.data.powder_sample()['data']
da_d_d_perp = da.transform_coords(["d", "d_perp"], graph=graph)
da_d_d_perp

In this case the `tof` dimension has not been renamed/replaced, since there are *two* output coordinates that depend on it.
We can bin this result into 2-D $d$ and $d_{\perp}$ bins and remove the `spectrum` and `tof` dimensions:

In [ ]:
da_d_d_perp.hist(d_perp=100, d=100).plot(norm="log")